In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from zipfile import ZipFile

In [ ]:
# Extract .zip file download links from webrobots.io and append to an interable.
url = 'https://webrobots.io/kickstarter-datasets/'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
zip_urls = []
for link in soup.find_all('a'):
    try:
        if 'Z.zip' in link.get('href'):
            zip_urls.append(link.get('href'))
    except:
        continue

In [59]:
# Define necessary directorys, loop through .zip file list created in step above, dowload .CSVs and append to df. 
path = Path().parent.absolute()
temp_zip = str(path)+'/temp.zip'

fdf = pd.DataFrame()
i=1

for zipurl in zip_urls:
    zipresp = urlopen(zipurl)
    tempzip = open(temp_zip, "wb")
    tempzip.write(zipresp.read())
    tempzip.close()
    zf = zipfile.ZipFile(temp_zip)
    
    text_files = zf.infolist()
    for text_file in text_files:
        try:
            df = pd.read_csv(zf.open(text_file.filename))
            df['pull_date'] = zip_file
            df = df[df['source_url'].str.contains('categories/games')]
            fdf = fdf.append(df)
        except:
            continue
    
    zf.close()
    print(i)
    i+=1

os.remove("temp.zip")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77


In [62]:
adf = fdf.drop(
         ['currency_symbol', 
          'currency_trailing_code',
          'current_currency',
          'disable_communication',
          'is_backing',
          'permissions',
          'urls',
          'unread_messages_count',
          'unseen_activity_count',
          'country_displayable_name',
          'projects',
          'total_hits',
          'seed',
          'search_url',
          'has_more',
          'last_update_published_at'
         ]
         , axis=1)

adf['created_at'] = pd.to_datetime(adf['created_at'].fillna(0).astype('int'),unit='s')
adf['deadline'] = pd.to_datetime(adf['deadline'].fillna(0).astype('int'),unit='s')
adf['launched_at'] = pd.to_datetime(adf['launched_at'].fillna(0).astype('int'),unit='s')
adf['state_changed_at'] = pd.to_datetime(adf['state_changed_at'].fillna(0).astype('int'),unit='s')

adf['pull_date'] = adf['pull_date'].str.replace('Kickstarter_', '').str.replace('Z.zip', '')
adf['pull_date'] = pd.to_datetime(adf['pull_date'], format='%Y-%m-%dT%H_%M_%S_%f')

/var/folders/mt/j3cd4z6111z5jqc6hx30y3k00000gn/T/ipykernel_1040/889761902.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  adf['pull_date'] = adf['pull_date'].str.replace('Kickstarter_', '').str.replace('Z.zip', '')


In [63]:
idx = adf.groupby(['slug'])['pull_date'].transform(max) == adf['pull_date']
adf = adf[idx]
adf = adf.drop_duplicates(subset=['slug'])

In [65]:
adf.to_csv(str(path)+'/KICK_Data.csv')